# jupyter-nbrequirements

<p style="font: 30px; text-transform: uppercase;">
    Jupyter Notebook dependency resolution and environment setup
</p>

---

<span style="font: 18px"><b>Description</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    This is an e2e pipeline from a single Jupyter notebook to fully set-up virtual environment ready to run the notebook.
    We're gonna demonstrate the whole functionality starting with setting notebook requirements, through Thoth configuration and dependency resolution to creating a complete virtual environment and setting a new Jupyter kernel.
    Hold tight. 
</p>

<span style="font: 18px"><b>Goal</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    Run all cells in the notebook.
</p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#_" data-toc-modified-id="_-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>_</a></span></li><li><span><a href="#Notebook-Content" data-toc-modified-id="Notebook-Content-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Notebook Content</a></span></li><li><span><a href="#Set-notebook-requirements" data-toc-modified-id="Set-notebook-requirements-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set notebook requirements</a></span></li><li><span><a href="#Get-notebook-requirements" data-toc-modified-id="Get-notebook-requirements-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get notebook requirements</a></span></li><li><span><a href="#Generate-Thoth-config" data-toc-modified-id="Generate-Thoth-config-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Generate Thoth config</a></span></li><li><span><a href="#Generate-Pipfile" data-toc-modified-id="Generate-Pipfile-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Generate Pipfile</a></span></li><li><span><a href="#Lock-down-dependencies" data-toc-modified-id="Lock-down-dependencies-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Lock down dependencies</a></span></li><li><span><a href="#Install-dependencies-into-virtual-environment" data-toc-modified-id="Install-dependencies-into-virtual-environment-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Install dependencies into virtual environment</a></span></li><li><span><a href="#Install-and-set-a-new-Jupyter-kernel" data-toc-modified-id="Install-and-set-a-new-Jupyter-kernel-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Install and set a new Jupyter kernel</a></span></li></ul></div>

---

## _

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext jupyter_require

<JupyterRequire.display.SafeScript object>

<JupyterRequire.display.SafeScript object>

---

In [3]:
%%requirejs

display = function (s, output) {
    output.append(`<pre>${JSON.stringify(s, null, 2)}</pre>`)
}

In [4]:
%%requirejs

execute_request = function(request, callbacks, options, context) {
    return new Promise( resolve => {
        const default_options = {
            silent: false,
            store_history: true,
            stop_on_error: true
        }

        if (_.isUndefined(context)) {
            context = {
                cell: Jupyter.notebook.get_executed_cell()
            }
        }

        let cell = context.cell

        options   = _.assign(default_options, options)
        callbacks = _.assign(cell.get_callbacks(), callbacks || {})

        let kernel = Jupyter.notebook.kernel

        console.debug(`Executing shell request:\n${request}\n\twith callbacks: `, callbacks)

        const msg_id = kernel.execute(request, callbacks, options)
        
        kernel.events.on("finished_iopub.Kernel", (e, d) => {
            if ( d.msg_id === msg_id ) {
                resolve()
            }
        })
    })
}
    

execute_shell_command = function(command, callbacks, options, context) {
    
    return execute_request(`!${command}`, callbacks, options, context)
}

execute_shell_script = function(command, callbacks, options, context) {
    
    return execute_request(`%%bash\n${command}`, callbacks, options, context)
}

execute_python_script = function(script, callbacks, options, context) {
    
    return execute_request(`${script}`, callbacks, options, context)
}

In [5]:
%%requirejs

dedent = function (strings, ...values) {
  const raw = typeof strings === "string" ? [strings] : strings.raw;

  // first, perform interpolation
  let result = "";
  for (let i = 0; i < raw.length; i++) {
    result += raw[i]
      // join lines when there is a suppressed newline
      .replace(/\\\n[ \t]*/g, "")
      // handle escaped backticks
      .replace(/\\`/g, "`");

    if (i < values.length) {
      result += values[i];
    }
  }

  // now strip indentation
  const lines = result.split("\n");
    
  let mindent = null;
  lines.forEach(l => {
    let m = l.match(/^(\s+)\S+/);
    if (m) {
      let indent = m[1].length;
      if (!mindent) {
        // this is the first indented line
        mindent = indent;
      } else {
        mindent = Math.min(mindent, indent);
      }
    }
  });

  if (mindent !== null) {
    const m = mindent; // appease Flow
    result = lines.map(l => l[0] === " " ? l.slice(m) : l).join("\n");
  }

  return result
    // dedent eats leading and trailing whitespace too
    .trim()
    // handle escaped newlines at the end to ensure they don't get stripped too
    .replace(/\\n/g, "\n");
}

indent = function(text, indent=4) {
    const indentation = " ".repeat(indent)
    
    return text
        .split("\n")
        .map((line) => `${indentation}${line}`)
        .join("\n")
}

In [6]:
%%requirejs

/**
 * Representation of source (Python index) for Python packages.
 */
Source = function(
    name =  "pypi",
    url  =  "https://pypi.org/simple",
    verify_ssl = true,
    warehouse,
    warehouse_api_url
) {
    this.name = name
    this.url  = url
    this.verify_ssl = verify_ssl
    this.warehouse = warehouse
    this.warehouse_api_url = warehouse_api_url
}

/**
 * Generate Pipfile entry for the given package.
 */
PackageVersion = function (
    name,
    version = "*",
    develop = false,
    index,
    hashes,
    markers,
    semantic_version,
    version_spec
) {
    this.name = name
    this.version = version
    this.develop = develop
    this.index   = index
    this.hashes  = hashes
    this.markers = markers
    this.semantic_version = semantic_version
    this.version_spec = version_spec
    
    this.to_pipfile = function() {
        let result = {}
        
        console.log("Generating Pipfile entry for package: ", this.name)

        if ( !_.isUndefined(this.index) )
            result["index"] = this.index.name

        if ( !_.isUndefined(this.markers) )
            result["markers"] = this.markers

        if ( _.isEmpty(result) ) {
            // Only version information is available.
            return { [this.name]: this.version }
        }

        result["version"] = this.version
        
        return { [this.name]: result }
    }
}


/**
 * Parse meta information stored in Pipfile or Pipfile.lock.
 */
Pipfile = function(packages = [], dev_packages = [], meta) {
    this.packages = packages
    this.dev_packages = dev_packages
    this.meta = meta
}

In [7]:
%%requirejs

DEFAULT_PYTHON_INDENT = 4

gather_library_usage = function (cells) {
    return new Promise(async (resolve) => {

        cells = cells || Jupyter.notebook.toJSON().cells

        console.log("Gathering requirements from cells, ", cells)

        cells.forEach( (c, i, a) => {
            const source = c.source
                .trim()
                .replace("?", "")  // remove Jupyter magic to display help
                .replace(/^[%!]{1}[^%]{1}.*$/gm, "\n")  // remove lines starting with single % or !
                .replace(/^\s*\n/gm, "")     // remove empty lines

            c.source = source
        })

        cells = cells.filter( c => (c.cell_type === "code") && (!c.source.startsWith("%%")) )

        let kernel = Jupyter.notebook.kernel
        
        notebook_content = cells.map(c => c.source).join("\n")
        notebook_content = indent(notebook_content, DEFAULT_PYTHON_INDENT * 3)

        console.debug("Notebook content: ", notebook_content)

        script = dedent(`
            import ast
            import distutils

            from pathlib import Path
            from invectio.lib import InvectioVisitor

            _STD_LIB_PATH = Path(distutils.sysconfig.get_python_lib(standard_lib=True))
            _STD_LIB = {p.name.rstrip(".py") for p in _STD_LIB_PATH.iterdir()}

            ast = ast.parse('''
            ${notebook_content}
            ''')

            visitor = InvectioVisitor()
            visitor.visit(ast)

            report = visitor.get_module_report()

            libs = filter(
                lambda k: k not in _STD_LIB | set(sys.builtin_module_names), report
            )

            list(libs)
        `)


        let callback = (msg) => {
            console.debug("Execution callback: ", msg)
            if (msg.msg_type === "error") {
                throw new Error(`Script execution error: ${msg.content.ename}: ${msg.content.evalue}`)
            }
            
            if (msg.msg_type === "stream") {
                console.log(msg.content.text)
            }
            else if ( msg.msg_type === "execute_result" ) {
                
                const result = msg.content.data["text/plain"].replace(/\'/g, '"')
                const requirements = JSON.parse(result)

                resolve(requirements)
            }
        }

        await execute_python_script(script, {iopub: {output: callback}})
    })
}

In [8]:
%%requirejs

Notebook = require("base/js/namespace").Notebook


Notebook.prototype.set_requirements = function(requirements, overwrite = true) {
    let metadata = Jupyter.notebook.metadata
    
    if ( _.isUndefined(metadata.requirements) || overwrite ) {
        metadata.requirements = requirements
    } else {
        console.debug("Requirements already exist. Updating.")
        // update the notebook metadata
        _.assign(metadata.requirements, requirements)
    }
        
    console.log("Notebook requirements have been set successfully.")
}


Notebook.prototype.get_requirements = function(ignore_metadata = false) {
    return new Promise(async (resolve) => {
        console.log("Reading notebook requirements.")
        
        let requirements = Jupyter.notebook.metadata.requirements

        if ( _.isUndefined(requirements) || ignore_metadata ) {
            console.log("Requirements are not defined.")

            requirements = gather_library_usage()
                .then((r) => {
                    let packages = {}
                    
                    r.forEach((p) => {
                        let package = new PackageVersion(p.toLocaleLowerCase())
                        
                        _.assign(packages, package.to_pipfile())
                    })
                
                    let kernel = Jupyter.notebook.kernel
                    let language_info = kernel.info_reply.language_info

                    const python_version = language_info.version
                    const requires = { python_version: python_version.match(/\d.\d/)[0] }

                    return { packages: packages, requires: requires, source: [new Source()]}
                })
                .catch(console.error)
            
            resolve(requirements)
        }

        resolve(requirements)
    })
}

In [9]:
%%requirejs

/**
 * Create Pipfile from notebook requirements
 */
create_pipfile = function(requirements, overwrite = false, sync = true) {
    return new Promise( async (resolve) => {
        if ( _.isUndefined(requirements) )
            requirements = await Jupyter.notebook.get_requirements()

        console.log("Creating Pipfile from notebook requirements.")

        let script = `
        import json

        from pathlib import Path
        from thoth.python import Pipfile

        if Path("Pipfile").exists() and "${overwrite}" != "true":
            raise FileExistsError("Pipfile already exists and \`overwrite\` is not set.")

        requirements: dict = json.loads("""${JSON.stringify(requirements)}""")
        pipfile = Pipfile.from_dict(requirements)
        pipfile.to_file()

        pipfile.to_string()
        `
        callback = (msg) => {
            console.debug("Execution callback: ", msg)
            if (msg.msg_type == "error") {
                throw new Error(`Script execution error: ${msg.content.ename}: ${msg.content.evalue}`)
            }

            console.log("Pipfile has been created successfully: ", msg.content.data["text/plain"])

            if (sync) {
                // sync notebook metadata with the Pipfile
                Jupyter.notebook.set_requirements(requirements)
                console.log("Notebook requirements have been synced with Pipfile.")
            }
            
            resolve(requirements)
        }

        await execute_python_script(script, {iopub: {output: callback}})
    })
}

In [10]:
%%requirejs

lock_requirements = function(requirements, sync = true) {
    return new Promise( async (resolve, reject) => {
        if ( _.isUndefined(requirements) ) 
            requirements = await Jupyter.notebook.get_requirements()

        // we want Pipfile to be synced with Pipfile.lock, always overwrite
        await create_pipfile(requirements, true)

        const command = `
            import json
            
            from thamos.lib import advise
            from thamos.cli import _load_pipfiles

            pipfile_content, pipfile_lock_content = _load_pipfiles()

            results = advise(
                pipfile_content,
                pipfile_lock_content,
                nowait=False,
                no_static_analysis=True  # static analysis is not yet functional for Jupyter NBs
            )

            if not results:
                raise Exception("Analysis was not successful.")

            result, error = results
            report = result["report"]

            if not error:
                pipfile = report[0][1]["requirements"]
                pipfile_lock = report[0][1]["requirements_locked"]
            else:
                raise Exception(f"Errors occured: {result}")

            json.dumps(pipfile_lock)
        `
        
        const timeout = setTimeout(() => {
            // interrupt the running script
            Jupyter.notebook.kernel.interrupt()
            
            reject(new Error("Timeout exceeded: Locking requirements was not successful."))
        }, 3000 * 60)
        
        let callback = (msg) => {
            console.debug("Execution callback: ", msg)
            
            if ( msg.msg_type == "error") {
                reject(new Error(`${msg.content.ename}: ${msg.content.evalue}`))
            }
            else if ( msg.msg_type == "stream" ) {  // adviser / pipenv log messages
                console.info(`[Thamos]: `, msg.content.text)
                
            }
            else if ( msg.msg_type == "execute_result" ){
                clearTimeout(timeout)
                
                const result = msg.content.data["text/plain"].replace(/(^')|('$)/g, "")
                const requirements_locked = JSON.parse(result)
                
                if (sync) {
                    // sync requirements locked with Pipfile.lock
                    Jupyter.notebook.set_requirements_locked(requirements_locked)
                    
                    console.log("Locked requirements have been synced with Pipfile.")
                }
                
                resolve(requirements_locked)
            }
            else
                reject(new Error(`Unknown message type: ${msg.msg_type}`))
        }

        await execute_python_script(command, {iopub: {output: callback}})
    })
}

In [99]:
%%requirejs

lock_requirements({
    'packages': {
        'pandas': '*',
    },
    'source': [new Source()]
}).then((requirements_locked) => {
    console.log("Locked requirements: ", requirements_locked)
    
    Jupyter.notebook.set_requirements_locked(requirements_locked)
})

In [11]:
%%requirejs

Notebook.prototype.set_requirements_locked = function(requirements_locked) {
    let metadata = Jupyter.notebook.metadata
    
    if ( _.isUndefined(metadata.requirements_locked) ) {
        metadata.requirements_locked = requirements_locked
    } else {
        console.debug("requirements_locked already exist. Updating.")
        // update the notebook metadata
        _.assign(metadata.requirements_locked, requirements_locked)
    }
        
    console.log("Notebook locked requirements have been set successfully.")
}

Notebook.prototype.get_requirements_locked = function(ignore_metadata = false, sync = true) {
    return new Promise(async (resolve, reject) => {
        console.log("Reading notebook locked requirements.")
        
        let requirements_locked = Jupyter.notebook.metadata.requirements_locked

        if ( _.isUndefined(requirements_locked) || ignore_metadata ) {
            console.log("Locked requirements are not defined.")

            lock_requirements(undefined, sync)
                .then( (r) => {
                    console.log("Successfully locked notebook requirements.", r)
                    resolve(r)
                })
                .catch( (err) => reject(new Error(err)) )
        } else {
            resolve(requirements_locked)
        }
    })
}

In [12]:
%%requirejs

utils = require("base/js/utils")

parse_console_output = function(str) {
    const ansi_re = /\x1b\[(.*?)([@-~])/g

    str = _.escape(str) + "\x1b[m"  // Ensure markup for trailing text

    let out = []
    let start = 0

    while ( (match = ansi_re.exec(str)) ) {
        let chunk = str.substring(start, match.index)

        if ( chunk ) out.push(chunk)

        start = ansi_re.lastIndex
    }
    
    const result = out.join("")

    return _.compose(utils.fixBackspace, utils.fixCarriageReturn)(result)
}

In [13]:
%%requirejs

/**
 * Ask Thoth for recommendations on application stack.
 */
create_pipfile_lock = function(requirements_locked, ignore_metadata = true, sync = true) {
   return new Promise( async (resolve, reject) => {
        if ( _.isUndefined(requirements_locked) ) {
            requirements_locked = await Jupyter.notebook.get_requirements_locked(
                ignore_metadata,
                sync
            )
        }

        console.log("Creating Pipfile.lock from notebook locked requirements.")

        let script = `
            import json

            requirements_locked = json.loads("""${JSON.stringify(requirements_locked, null, 4)}""")

            with open("Pipfile.lock", "w") as f:
                json.dump(requirements_locked, f, sort_keys=True, indent=4)
        `
        
        callback = (msg) => {
            console.debug("Execution callback: ", msg)
            if (msg.content.status == "error") {
                reject(new Error(`Script execution error: ${msg.content.ename}: ${msg.content.evalue}`))
            }

            resolve()
        }

        await execute_python_script(script, {shell: {reply: callback}})
    }) 
}

In [14]:
%%requirejs

install_requirements = function(
    requirements,
    dev_packages = false,
    pre_releases = false
) {
    return new Promise( async (resolve, reject) => {
        
        requirements = requirements || []
        
        /**
         * Logging callback
         */
        let iopub_callback = (msg) => {
            console.debug("Execution logging callback: ", msg)

            if ( msg.msg_type == "error") {
                reject(new Error(`${msg.content.ename}: ${msg.content.evalue}`))
            }

            else if ( msg.msg_type == "stream" ) {  // adviser / pipenv log messages
                const stream = msg.content.name || "stdout"
                const text = parse_console_output(msg.content.text)

                if ( stream === "stderr" ) {
                    console.warn(`[pipenv]: `, text)
                } else
                    console.log(`[pipenv]: `, text)

            }
        }
        
        /**
         * Execution done callback
         */
        let shell_callback = (msg) => {
            console.debug("Execution shell callback: ", msg)
            
            if ( msg.content.status == "error") {
                reject(new Error(`${msg.content.ename}: ${msg.content.evalue}`))
            }
            
            console.log("Requirements have been successfully installed")
            
            resolve()
        }
        

        let opts = ""
        
        if ( dev_packages) opts += "--dev "
        if ( pre_releases) opts += "--pre"
        
        console.log("Installing requirements: ")
        
        await execute_shell_command(
            `pipenv install --ignore-pipfile --keep-outdated ${opts} ${requirements}`,
            {iopub: {output: iopub_callback}, shell: {reply: shell_callback}}
        )
    })
}

In [15]:
%%requirejs

install_kernel = function(name) {
    return new Promise( async (resolve, reject) => {
        
        /**
         * Logging callback
         */
        let iopub_callback = (msg) => {
            console.debug("Execution logging callback: ", msg)

            if ( msg.msg_type == "error") {
                reject(new Error(`${msg.content.ename}: ${msg.content.evalue}`))
            }

            else if ( msg.msg_type == "stream" ) {  // adviser / pipenv log messages
                const stream = msg.content.name || "stdout"
                const text = parse_console_output(msg.content.text)

                if ( stream === "stderr" ) {
                    console.warn(`[pipenv]: `, text)
                } else
                    console.log(`[pipenv]: `, text)

            }
        }
     
        const kernel_name = name || Jupyter.notebook.notebook_name
            .replace(".ipynb", "")
            .replace(/\s+/g, "_")
        
        // check if ipython and ipykernel are both installed
        const script = dedent`
            PACKAGE_LIST=$(pipenv run pip list | cut -d' ' -f 1)

            if [[ $(echo $PACKAGE_LIST | grep -E "ipython\$|ipykernel\$" | wc -l) != 2 ]]; then
                echo "Packages 'ipython' and 'ipykernel are already installed'"
            else
                echo "Installing required packages: 'ipython', 'ipykernel'"
                pipenv run pip install ipython ipykernel
            fi
        `
        
        console.log(`Installing kernel ${kernel_name}.`)
        
        await execute_shell_script(script, {iopub: {output: iopub_callback}})
            .then( async () => {
                return execute_shell_command(
                    `pipenv run ipython kernel install --user --name=${kernel_name}`,
                    {iopub: {output: iopub_callback}}
                )
            })
            .catch(reject)

        console.log(`Kernel '${kernel_name}' has been installed.`)
        
        resolve(kernel_name)
    })
}

load_kernel = function(name) {
    return new Promise( async (resolve, reject) => {
        const kernel_name = name || Jupyter.notebook.notebook_name
            .replace(".ipynb", "")
            .replace(/\s+/g, "_")
        
        const kernel_selector = Jupyter.notebook.kernel_selector
        
        kernel_selector.request_kernelspecs()
        
        let wait_for_kernelspec = () => {
            if ( _.has(kernel_selector.kernelspecs, name) )
                resolve(name)
            else
                setTimeout(wait_for_kernelspec, 50)  // check again in 50ms
        }
        
        setTimeout(() => reject(
            new Error(`Timeout exceeded while waiting for kernel spec: ${name}`)
        ), 1000)
        
        wait_for_kernelspec()
    })
}

set_kernel = function(name) {
    return new Promise( async (resolve, reject) => {
        const kernel_name = name || Jupyter.notebook.notebook_name
            .replace(".ipynb", "")
            .replace(/\s+/g, "_")
        
        const kernel_selector = Jupyter.notebook.kernel_selector
        
        console.log(`Setting kernel: ${kernel_name}.`)

        if ( kernel_selector.current_selection === kernel_name ) {
            console.log(`Kernel ${kernel_name} is already set.`)
            return resolve()
        }

        // make sure kernelspec exists
        if ( !_.has(kernel_selector.kernelspecs, kernel_name) ) {
            return reject(new Error(`Missing kernel spec: ${kernel_name}`))
        }
        
        kernel_selector.set_kernel(kernel_name)
        
        resolve(kernel_name)
    })
}

In [ ]:
%%requirejs

install_kernel()
    .then( (name) => load_kernel(name))
    .then( (name) => {
        console.log(`Kernel spec '${name}' is ready.`)
        set_kernel(name)
    })
    .then( (name) => console.log(`Kernel '${name}' has been set.`))

---

## Notebook Content

For the purposes of this demo, let's pretend that all the notebook source code is actually contained in this section.

In [17]:
import json
import sys

import pandas as pd
import sklearn

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("<path>")
df.head()

with open("requirements.json") as f:
    requirements = json.load(f)

try:
    ...
except Exception as exc:
    print(exc, file=sys.stderr)

---

Load the `nbrequirements` magic

In [16]:
import sys;

sys.path.insert(0, "../../")  # jupyter-nbrequirements
sys.path.insert(0, "/home/macermak/RedHat/adviser")  # patched aviser

%load_ext jupyter_nbrequirements

---

## Set notebook requirements

<p style="text-align: justify; text-justify: inter-word;">
    The goals are in order of operations that we want to perform when setting up the environment.
    The first step is to define notebook requirements.
</p>

<span style="font: 18px"><b>Acceptance Criteria</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    The notebook has requirements embedded in its metadata
</p>

<span style="font: 18px"><b>How to do it</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    There is a cell magic command <code>%%requirements</code> which takes the content of a cell and turns it into notebook requirements.
</p>
    
Example:

    %%requirements
    
    [[source]]
    name = "pypi"
    url = "https://pypi.org/simple"
    verify_ssl = true

    [dev-packages]
    autopep8 = "*"

    [packages]
    ipython = "*"
    ipykernel = "*"

    [requires]
    python_version = "3.6"


In [ ]:
%%requirements

[[source]]
name = "pypi"
url = "https://pypi.org/simple"
verify_ssl = true

[dev-packages]
autopep8 = "*"

[packages]
ipython = "*"
ipykernel = "*"

[requires]
python_version = "3.6"

---

## Get notebook requirements

<p style="text-align: justify; text-justify: inter-word;">
    Now suppose that we've received the notebook from somebody else. We want to check what requirements the notebook has defined and eventually, what are the <i>real</i> notebok requirements.
</p>

<span style="font: 18px"><b>Acceptance Criteria</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    We can safely check what requirements are defined and which are actually used.
</p>

<span style="font: 18px"><b>How to do it</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    There is a line magic command <code>%requirements</code> which displays the content of notebok requirements metadata, and if it doesn't exist, it performs static analysis and checks for library usage in the notebook source code.
</p>

Example:

    %requirements  # notice the single % sign


In [19]:
%requirements

[packages]
ipython = "*"
pandas = "*"

[dev-packages]

[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[requires]
python_version = "3.6"




<p style="text-align: justify; text-justify: inter-word;">
    Now, the user who specified the notebook requirements had probably had a tough night the day before, so he did not get them all.
    <br>
    Let's ... well ... ignore it ... and gather them ouselves!
</p>

In [22]:
%requirements --ignore-metadata

[packages]
pandas = "*"
ipython = "*"

[dev-packages]

[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[requires]
python_version = "3.6"




Few remarks about the output that we see above. If you take a look at the [#Code](#Code) section, you'll see that the imports actually look like this:

```
import json
import sys

import pandas as pd
import sklearn
```

So why do we *NOT* see all of these requirements in the output?

First of all, `json` and `sys` are somewhat special, `json` is a part of **standard library** and `sys` is a **built-in** module. Which means that they don't need to be installed.

As far as `sklearn` is concerned, we don't use it in this notebook. That's right, we track not only **imports**, but also the **usage**.

---

## Generate Thoth config

<p style="text-align: justify; text-justify: inter-word;">
    Thoth uses configuration file which looks something like this:
    
    # A remote Thoth service to talk to:
    host: stage.thoth-station.ninja

    # Configure TLS verification for communication with remote Thoth instance:
    tls_verify: true

    # Format of requirements file, currently supported is only Pipenv:
    requirements_format: pipenv

    runtime_environments:
      - name: 'fedora:30'
        # Operating system for which the recommendations should be created:
        operating_system:
          name: fedora
          version: '30'
        # Hardware information for the recommendation engine:
        hardware:
          # Intel(R) Core(TM) i7-6600U CPU @ 2.60GHz
          cpu_family: 6
          cpu_model: 78
        # Software configuration of runtime environment:
        python_version: '3.6'
        cuda_version: null
        # Recommendation type - one of testing, stable, latest:
        recommendation_type: stable
        # Number of latest versions considered during advises.
        limit_latest_versions: null

    #
    # Configuration of bots:
    #
    managers:
      - name: pipfile-requirements
      - name: info
      - name: version
        configuration:
          # A list of maintainers (GitHub or GitLab accounts) of this repository:
          maintainers: []
          # A list of assignees to which the opened pull requests and issues should
          # be assigned to:
          assignees: []
          # Labels for issues and pull requests:
          labels:
            - bot
          # Automatically maintain a changelog file stating features of new
          # releases:
          changelog_file: true
</p>
<p style="text-align: justify; text-justify: inter-word;">
    In order to be able to fully configure Thoth functionality, we would like to be able to simply generate the file.
</p>

<span style="font: 18px"><b>Acceptance Criteria</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    The <code>.thoth.yaml</code> file has been generated
</p>

<span style="font: 18px"><b>How to do it</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    There is a subcommand to the <code>%requirements</code> command called <code>config</code>. This generates the default <code>.thoth.yaml</code> config file.
</p>
    
Example:

    %requirements config


In [ ]:
%requirements config --to-file

Now let's leave the config as-is except for a simple change... let's disable `tls_verify` and set `fedora:29` as our operating system.

In [105]:
# set tls_verify to false
!perl -i -pe 's/(?<=tls_verify: )(false|true)/false/' .thoth.yaml
# set operating system version to 29
!perl -i -pe "s/(?<=version: )('30')/'29'/" .thoth.yaml
# set name to fedora:29
!perl -i -pe "s/fedora:30/fedora:29/" .thoth.yaml

In [106]:
%requirements config

# This is  Thoth's configuration file placed in a root of a repo
# (named as .thoth.yaml) used by Thamos CLI as well as by Thoth bots. Please
# adjust values listed below as desired.

# A remote Thoth service to talk to:
host: stage.thoth-station.ninja

# Configure TLS verification for communication with remote Thoth instance:
tls_verify: false

# Format of requirements file, currently supported is only Pipenv:
requirements_format: pipenv

runtime_environments:
  - name: 'fedora:29'
    # Operating system for which the recommendations should be created:
    operating_system:
      name: fedora
      version: '29'
    # Hardware information for the recommendation engine:
    hardware:
      # Intel(R) Core(TM) i7-6600U CPU @ 2.60GHz
      cpu_family: 6
      cpu_model: 78
    # Software configuration of runtime environment:
    python_version: '3.6'
    cuda_version: null
    # Recommendation type - one of testing, stable, latest:
    recommendation_type: stable
    # Number of latest v

---

## Generate Pipfile

<p style="text-align: justify; text-justify: inter-word;">
    Now we're getting to the part in which we want to work with the requirements -- that is, install them -- and in order to do that, we need a manifest file. In our case, it's gonna be the Pipfile.
</p>

<span style="font: 18px"><b>Acceptance Criteria</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    The <b>Pipfile</b> has been generated from notebook requirements.
</p>

<span style="font: 18px"><b>How to do it</b></span><br>
    The <code>%requirements</code> magic has an option <code>--to-file</code> which outputs the requirements to the Pipfile.

<p style="text-align: justify; text-justify: inter-word;">
</p>
    
Example:

    %requirements --to-file


For the purpose of this example -- check that there is no Pipfile present

In [ ]:
%cat Pipfile

In [ ]:
%requirements --ignore-metadata --to-file

And now ...

In [ ]:
%cat Pipfile

---

## Lock down dependencies

<p style="text-align: justify; text-justify: inter-word;">
Here we're getting to the core part. We want to resolve the software stack and lock down dependencies so that the software stack is <i>thoth-optimal</i>. 
</p>

<span style="font: 18px"><b>Acceptance Criteria</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    The Pipfile.lock has been generated using Thoth adviser API.
</p>

<span style="font: 18px"><b>How to do it</b></span><br>


<p style="text-align: justify; text-justify: inter-word;">
    The <code>%requirements</code> magic has a subcommand <code>lock</code> which takes an optional parameter <code style="white-space: nowrap;">--backend</code></span>. It triggers an analysis in <i>thoth-backend-stage</i> namespace and outputs the resolved software stack with locked down dependencies to the <b>Pipfile.lock</b>. 
</p>
    
Example:

    %requirements lock
    

Once again, check that Pipfile.lock is not present

In [23]:
%cat Pipfile.lock

cat: Pipfile.lock: No such file or directory


And also ignore the notebook metadata, just in case the developer also locked requirements (and remember, he had a tough night...)

In [27]:
%requirements lock --ignore-metadata

The requirements are cached, so when we want to output them to a file, we don't need to go through the analysis again.

In [153]:
%requirements lock --ignore-metadata --to-file

---

## Install dependencies into virtual environment

<p style="text-align: justify; text-justify: inter-word;">
</p>

<span style="font: 18px"><b>Acceptance Criteria</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
</p>

<span style="font: 18px"><b>How to do it</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
</p>
    
Example:

    %requirements install


In [ ]:
%requirements install

---

## Install and set a new Jupyter kernel

<p style="text-align: justify; text-justify: inter-word;">
</p>

<span style="font: 18px"><b>Acceptance Criteria</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
</p>

<span style="font: 18px"><b>How to do it</b></span><br>

<p style="text-align: justify; text-justify: inter-word;">
    
</p>
    
Example:
    
    %requirements kernel install [name]
    
    # if we want to also set a kernel
    # by default, this assumes a kernel matching the name of your notebook,
    # optionally you can provide custom name of an existing kernelspec
    %requirements kernel set [name]


Lets check our current kernel specification

In [ ]:
%requirements kernel

Now install the new kernel from the notebook requirements

In [ ]:
%requirements kernel install

And at the very end of this demo ... set the new Jupyter kernel.

First check out the current kernel spec (agian, just to demonstrate that there is no `example` kernel present)

> HINT: The kernels are located at the toolbar menu: *Kernel* -> *Change kernel*

If it is the case and there is already such kernel, feel free to provide custom kernel name by `%requirements kernel set <name>`

> WARNING: After you issue this command, you're gonna have a fresh kernel ready, so don't expect your variables or imports to be present

In [ ]:
%requirements kernel set

Remember the error that we got about `pandas` library not being present?

In [ ]:
from IPython.core.display import HTML

try:
    import pandas as pd  # <--- This was not possible before
    
    display(HTML("""
        <br>
        <div style="display: grid">
            <img style="margin: 0 auto;" src="/static/base/images/logo.png?v=641991992878ee24c6f3826e81054a0f" alt="Jupyter Notebook">
        </div>
        <hr>
        <center><p style="text-align: center; font-size: 21px"> Thank you for your attention! </p></center>
    """))
except:
    
    display(HTML("""
        <br>
        <div style="display: grid">
            <i class="fas fa-ban" style="margin: 0 auto; font-size: 80px; color: red;"></i>
        </div>
        <hr>
        <center><p style="text-align: center; font-size: 21px"> Time to blame the QA... </p></center>
    """))